In [1]:
import pandas as pd
from scrapy.http import TextResponse
import requests
import datetime

In [2]:
title_ls,link_ls = [],[]
content_str = 0
for page in range(1,30+1):
    url = 'https://m.news.naver.com/newsflash.nhn?mode=LS2D&sid1=101&sid2=259&page={}'.format(page)
    req = requests.get(url)
    response = TextResponse(req.url , body=req.text , encoding='utf-8')
    for content in range(1,20+1):
        try: 
            content_str += 1
            title_ls.append(response.xpath('//*[@id="newsflash{}"]/a/div/span[1]/strong/text()'.format(content_str)).extract()[0])
            link_ls.append(response.xpath('//*[@id="newsflash{}"]/a/@href'.format(content_str)).extract()[0])
        except : print(page,content)

In [3]:
len(title_ls) == len(link_ls)

True

In [4]:
df = pd.DataFrame()
df['title'] = title_ls
df['link'] = link_ls
df['link'] = ['https://m.news.naver.com' + i for i in df['link']]
df.tail()

,title,link
595,"이주열 韓銀총재, BIS 신임이사 선출",https://m.news.naver.com/read.nhn?oid=016&aid=...
596,"은행대출, 각종 우대금리까지 ‘깨알공개’",https://m.news.naver.com/read.nhn?oid=016&aid=...
597,"농협금융, 16일 자회사 CEO 임추위 개최",https://m.news.naver.com/read.nhn?oid=277&aid=...
598,"한은, 통화금융대책반 회의 개최…美 주가 급락 영향 점검",https://m.news.naver.com/read.nhn?oid=119&aid=...
599,"한-아세안 청년사업가, 한화생명 ‘드림플러스 강남’ 방문",https://m.news.naver.com/read.nhn?oid=018&aid=...


In [5]:
content_ls = []

for i in df['link'].values:
    req = requests.get(i)
    response = TextResponse(req.url , body=req.text , encoding='utf-8')
    content_ls.append(','.join(response.xpath('//*[@id="dic_area"]/text()').extract()).replace(',',''))

In [6]:
len(title_ls) == len(link_ls) == len(content_ls)

True

In [7]:
df['content'] = content_ls
df['content'] = [i.replace("\t",'').replace('\n','') for i in df['content'].values]
df.tail()

,title,link,content
595,"이주열 韓銀총재, BIS 신임이사 선출",https://m.news.naver.com/read.nhn?oid=016&aid=...,이주열<> 한국은행 총재가 11일(현지시간) 스위스 바젤 국제결제은행(BIS) 본부...
596,"은행대출, 각종 우대금리까지 ‘깨알공개’",https://m.news.naver.com/read.nhn?oid=016&aid=...,은행이 가계에 돈을 빌려줄 때 지점장 전결이나 본부 차원에서 적용하는 우대금리가 공...
597,"농협금융, 16일 자회사 CEO 임추위 개최",https://m.news.naver.com/read.nhn?oid=277&aid=...,[아시아경제 박소연 기자] 김광수 NH농협금융지주 회장이 연말 자회사 최고경영자(C...
598,"한은, 통화금융대책반 회의 개최…美 주가 급락 영향 점검",https://m.news.naver.com/read.nhn?oid=119&aid=...,한국은행은 13일 윤면식 부총재 주관으로 통화금융대책반 회의를 열고 미국 주가 하락...
599,"한-아세안 청년사업가, 한화생명 ‘드림플러스 강남’ 방문",https://m.news.naver.com/read.nhn?oid=018&aid=...,[이데일리 유재희 기자] 한화생명 ‘드림플러스 강남’에 아세안 국가의 청년 사업가들...


In [8]:
df.to_csv('{}_with_link_naver.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False),index=False)